In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("joint_data_trimmed.csv")

In [3]:
df.head()

,Unnamed: 0,subid,trialid,itemid,cond,sentnum,ianum,ia,blink,skip,...,singlefix.sac.in,singlefix.sac.out,singlefix.launch,singlefix.land,singlefix.cland,singlefix.dur,lang,trial,supplementary_id,uniform_id
0,1,du_01,1.0,1.0,1.0,1.0,1.0,Janus,1.0,0.0,...,NaN,15.0,NaN,0.0,-3.0,154.0,du,NaN,DU_01,du_1
1,2,du_01,1.0,1.0,1.0,1.0,2.0,is,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,du,NaN,DU_01,du_1
2,3,du_01,1.0,1.0,1.0,1.0,3.0,in,0.0,0.0,...,7.0,6.0,6.0,1.0,-0.5,551.0,du,NaN,DU_01,du_1
3,4,du_01,1.0,1.0,1.0,1.0,4.0,de,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,du,NaN,DU_01,du_1
4,5,du_01,1.0,1.0,1.0,1.0,5.0,oude,1.0,0.0,...,15.0,-13.0,14.0,1.0,-1.5,189.0,du,NaN,DU_01,du_1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855123 entries, 0 to 855122
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           855123 non-null  int64  
 1   subid                855123 non-null  object 
 2   trialid              855122 non-null  float64
 3   itemid               855122 non-null  float64
 4   cond                 855122 non-null  float64
 5   sentnum              855122 non-null  float64
 6   ianum                855122 non-null  float64
 7   ia                   854741 non-null  object 
 8   blink                855122 non-null  float64
 9   skip                 855122 non-null  float64
 10  nrun                 639530 non-null  float64
 11  reread               639530 non-null  float64
 12  nfix                 639530 non-null  float64
 13  refix                639454 non-null  float64
 14  reg.in               639530 non-null  float64
 15  reg.out          

In [5]:
df.describe()

,Unnamed: 0,trialid,itemid,cond,sentnum,ianum,blink,skip,nrun,reread,...,firstfix.cland,firstfix.dur,singlefix,singlefix.sac.in,singlefix.sac.out,singlefix.launch,singlefix.land,singlefix.cland,singlefix.dur,trial
count,855123.000000,855122.000000,855122.000000,855122.0,855122.000000,855122.000000,855122.000000,855122.000000,639530.000000,639530.000000,...,639530.000000,639530.000000,855122.000000,492301.000000,495066.000000,492301.000000,496614.000000,496614.000000,496614.000000,0.0
mean,427562.000000,6.319812,6.402988,1.0,5.100584,84.710652,0.101174,0.252118,1.456685,0.315846,...,-0.913472,214.771812,0.580752,0.879785,-1.286536,-1.957609,2.826402,-0.689971,216.742861,NaN
std,246852.891458,3.440210,3.458831,0.0,2.697842,51.443266,0.301559,0.434229,0.831255,0.464852,...,2.191979,94.834265,0.493436,25.398166,25.870237,25.378360,2.138574,1.960071,94.943364,NaN
min,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,-15.000000,2.000000,0.000000,-112.000000,-112.000000,-117.000000,0.000000,-14.000000,2.000000,NaN
25%,213781.500000,3.000000,3.000000,1.0,3.000000,41.000000,0.000000,0.000000,1.000000,0.000000,...,-2.000000,156.000000,0.000000,4.000000,-3.000000,1.000000,1.000000,-2.000000,158.000000,NaN
50%,427562.000000,6.000000,6.000000,1.0,5.000000,82.000000,0.000000,0.000000,1.000000,0.000000,...,-1.000000,200.000000,1.000000,8.000000,7.000000,5.000000,3.000000,-0.500000,203.000000,NaN
75%,641342.500000,9.000000,9.000000,1.0,7.000000,124.000000,0.000000,1.000000,2.000000,1.000000,...,0.500000,255.000000,1.000000,11.000000,10.000000,8.000000,4.000000,0.500000,257.000000,NaN
max,855123.000000,12.000000,13.000000,1.0,16.000000,243.000000,1.000000,1.000000,16.000000,1.000000,...,18.000000,12688.000000,1.000000,109.000000,106.000000,104.000000,23.000000,18.000000,12688.000000,NaN
